# Machine learning models to predict demographic parameters

*Demographic history* of populations is the history of population development that includes such events as changes of population size, split time, migration rates and so on. Such history leaves traces in the genomes of individuals and it is possible to reconstruct this record from genetic data using different statistical and algorithmic approaches. The reconstruction of demographic history is called *demographic inference*.

There are several methods for demographic inference, for example, dadi and moments to name some of them. Those tools are very popular and are based on the same statistic of the genetic data - *allele frequency spectrum*. Demographic inference for the whole genome data is a computationally complex problem. Therefore, different statistics are built from it and used for the inference. Allele frequency spectrum is one of the most popular among such statistics. It is a histogram of mutation frequencies. For two populations it is a two dimensional matrix $A$, where entry $A[i, j]$ corresponds to the number of positions where the mutation occurred in $i$ individuals of the first population and in $j$ individuals of the second population. Allele frequency spectrum of two populations is usually pictured as a heatmap.

Tools for demographic inference like dadi and moments usually have two components: simulation and optimization. Simulation component provides an opportunity to simulate a statistic under the proposed demographic history and measure a value of similarity, namely likelihood, between simulation and real data. Optimization component takes a parameterized model of demographic history as input and searches for the parameters of this model that will give the best value likelihood with the data. Unfortunately, optimization implemented in dadi and moments sometimes is not efficient in practice.

GADMA was presented as a new tool for efficient unsupervised demographic inference. It uses the simulation components of existing tools for demographic inference and proposes new optimization based on the genetic algorithm. It also avoids model specification by the researcher. Instead the researcher could choose the *structure* of the demographic history - number of time epochs before and between split events. For two populations it is a list of two numbers: number of epochs before the ancestral population divergence and number of epochs after it. GADMA infers all possible parameters for the given structure using a global search genetic algorithm. It avoids human bias in model construction.


We decided to **speed up the genetic algorithm** by using prediction models from machine learning. We will do it in the most common case: demographic inference of **two populations**. The **idea** is to train some machine learning models and use them for initial estimations of demographic parameters in GADMA. We are going to train and test two models: **Random Forest regression** and **Convolutional Neural Networks**.

Some details: as a data for our prediction we will use allele frequency spectrum. We will use dadi and moments. Our models will predict demographic parameters for the structure (2,1) with most diverse set of demographic parameters. It is not recommended to use GADMA with structure more than (2,1). If the actual structure in demographic inference will differ then those parameters will be transformed using GADMA algorithms to fit it.

## Install libraries

In [ ]:
!pip install pandas
!pip install dadi
!pip install gadma==2.0.0rc18
!pip install demes
!pip install git+https://bitbucket.org/simongravel/moments.git

## Generate dataset

This section generates datasets that will be used for training and validation.
Totally we will generate **200,000** allele frequency spectra.

We want to train our models to predict parameters for the demographic history with (2,1) structure. we will use simulated allele frequency spectra using moments. The first 100,000 parameters are generated using special distribution from GADMA that provides more close to real values, the latter 100,000 parameters are generated uniformly.

![title](model_pictures/model_with_structure_2_1.png)

#### 1) Create demographic model with structure 2,1 using GADMA

In [1]:
import scipy.stats as sts
import pylab
import moments
import gadma
import numpy as np
import pandas as pd
import tqdm

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

warning in stationary: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


In [2]:
# Create demographic model with structure 2,1 using gadma.

model_2_1 = gadma.StructureDemographicModel(
    initial_structure=[2,1],
    final_structure=[2,1],
    has_migs=True,
    has_sels=False,
    has_dyns=True,
    sym_migs=False,
    frac_split=True,
    migs_mask=None,
    )

# We also create demographic model with structure 1,1 as we will need it.
model_1_1 = gadma.StructureDemographicModel(
    initial_structure=[1,1],
    final_structure=[1,1],
    has_migs=True,
    has_sels=False,
    has_dyns=True,
    sym_migs=False,
    frac_split=True,
    migs_mask=None,
    )

#### 2) Choose the size of allele frequency spectrum

The size of spectrum should not be too large as we always can project AFS down but we cannot project it up. So we choose it equal to 5 chromosomes per population. 

In [3]:
n_pop = 2
pop_labels = ["Pop 1", "Pop 2"]

ns_per_pop = 5
ns = [ns_per_pop for _ in range(n_pop)]

#### 3) Simulations and creation of file with sample data

Here we generate 100,000 sets of parameters and save them in `labels.txt` file. Then for each set of parameters allele frequency spectrum is generated with moments and saved to `data.txt` file.

In [4]:
n_samples = 100000
y_file = r"labels.txt"
x_file = "data.txt"

In [ ]:
# 1. Generate points using distribution from GADMA
y_cur = [[var.resample() for var in model_2_1.variables] for _ in range(n_samples)]

# 2. For better training we fix values so they match 1,1 structure for the one third
inds = np.random.choice(range(n_samples), size=n_samples//3, replace=False)
for ind in inds:
    y_cur[ind][1] = 1.0  # nu11
    #y_cur[ind][2] = "Sud"  # dyn11
df = pd.DataFrame(y_cur, dtype=object)

# 3. Save labels - true predictions
np.savetxt(y_file, df.values, fmt="%s")

# 4. Simulate AFS for each set of parameters and save it as lines in file
engine = gadma.get_engine("moments")
engine.model = model_2_1
open(x_file, 'w').close()
for i, u in tqdm.tqdm(enumerate(y_cur)):
    popt = u.tolist()
    data = engine.simulate(values=popt, ns=ns, sequence_length=None, population_labels=pop_labels)
    data = data.fold()

    data.to_file(f'__fs_data.txt')
    with open('__fs_data.txt', 'r') as f:
        for i in range(1):
            f.readline()
        x = f.readline()
        with open(x_file, 'r+') as f:
            f.seek(0, 2) 
            f.write(x)

## Train machine learning models

### Import tools

In [5]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import RegressorChain

from sklearn.model_selection import train_test_split as tr
from sklearn.model_selection import RandomizedSearchCV, RepeatedKFold, cross_val_score, cross_validate
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn import metrics

import joblib

import matplotlib.pyplot as plt 
import matplotlib.ticker as ticker
#from matplotlib.backends.backend_pdf import PdfPages

from scipy.optimize import root
from scipy.stats import randint, uniform
from math import sqrt, exp, sin

import os
from tqdm import tqdm
import pickle
import random
import numpy as np

#### Read data from files

In [6]:
def prepare_spectrum_for_input(spectrum):
    # We remove zero element and normalize spectrum
    spectrum /= np.sum(spectrum)
    return spectrum

def dynamics2classes(y, back=False):
    # change strings to classes
    _map = [("Sud", '0'), ("Lin", '1'), ("Exp", '2')]
    dyn2cls = {dyn: cls for dyn, cls in _map}
    cls2dyn = {cls: dyn for dyn, cls in _map}
    y_fixed = []
    for _y in y:
        _y = np.array(_y)
        if not back:
            transf_dict = dyn2cls
        else:
            transf_dict = cls2dyn
        for key in transf_dict:
            _y[_y == key] = transf_dict[key]
        y_fixed.append(_y)
    return y_fixed

def read_labels(filename):
    with open(filename, 'r') as f:
        y = []
        for line in f:
            values = [float(val) if not isinstance(var, gadma.DynamicVariable) else val
                  for val,var in zip(line.split(), model_2_1.variables)]
            y.append(values)
    y = dynamics2classes(y)
    y = np.array(y, dtype=object)
    return y

def read_data(filename):
    with open(filename, 'r') as f:
        X = []
        i = 0
        for line in f:
            assert len(line.split()) == (ns_per_pop+1)**2, f"{i} {line} {len(line.split())}"
            spectrum = np.array(line.split(), dtype=float)
            spectrum = prepare_spectrum_for_input(spectrum=spectrum)
            X.append(spectrum)
            i += 1
    return X

In [7]:
X_total = read_data(x_file)
y_total = read_labels(y_file)
n_sample = min(len(X_total), len(y_total))
print(f"Number of read samples: {n_sample}")
X_total = X_total[:n_sample]
y_total = y_total[:n_sample]

Number of read samples: 100000


### Create train and test samples

In [8]:
X_train, X_test, y_train, y_test = tr(X_total, y_total, train_size = 0.8, random_state=500, shuffle=False)

## Random Forest

#### Project down train dataset

As it turned out from our experiments Random Forest performs better with small dataset. It also important that they are from the first half that was generated with special distribution.

In [9]:
RF_n_sample = 2000

X_train_RF = X_train[:RF_n_sample]
y_train_RF = y_train[:RF_n_sample]

#### Prepare function to print measure of fitness and load one test example

In [10]:
def print_measures(y_true, y_pred):
    """
    Returns MSE for all continous parameters and accuracy for all discrete.
    """
    is_cont = np.array([not isinstance(var, gadma.DynamicVariable) for var in model_2_1.variables])
    is_disc = np.array([not cont for cont in is_cont])
        
    cont_y_pred = np.array([np.array(_y)[is_cont] for _y in y_pred], dtype=float)
    disc_y_pred = np.array([np.array(_y)[is_disc] for _y in y_pred])
        
    cont_y_true = np.array([np.array(_y)[is_cont] for _y in y_true], dtype=float)
    disc_y_true = np.array([np.array(_y)[is_disc] for _y in y_true])
        
    mse = metrics.mean_squared_error(y_true=cont_y_true, y_pred=cont_y_pred)
        
    acc = []
    for i in range(disc_y_true.shape[1]):
        acc.append(metrics.accuracy_score(y_true=disc_y_true[:, i], y_pred=disc_y_pred[:, i]))
        
    print("Mean squared error:", mse)
    print("Mean accuracy score:", np.mean(acc))

In [11]:
# We have AFS for a special case. We know the best value of likelihood and best parameters.
data = moments.Spectrum.from_file("fs_data.fs")
best_ll = -1310.931
best_params = {
    't1': 1.0,
    'nu11': 1.0,
    'dyn11': 'Sud',
    's1': 0.5,
    't2': 0.05,
    'nu21': 1.0,
    'nu22': 0.1,
    'dyn21': 'Sud',
    'dyn22': 'Sud',
    'm2_12': 5.0,
    'm2_21': 2.5,
}
engine = gadma.get_engine("moments")
engine.model = model_2_1
engine.data = data
best_ll = engine.evaluate(best_params)

# we need to prepare data as input to ML model
test_x = data.project(ns).fold()
test_x = np.array(test_x).flatten()
test_x /= np.sum(test_x)

### Model 1. Independent parameter prediction

We will start with simple Random forest with independent predictions for each demographic parameter.

![title](model_pictures/model_1_scheme.png)

In [12]:
class MyRFFor21ModelIndependent(object):
    def __init__(self):
        # We create our models
        # We have RF regressors for continous variables and Classifiers for dynamics
        self.models = {}
        for var in model_2_1.variables:
            if not isinstance(var, gadma.DynamicVariable):
                self.models[var.name] = RandomForestRegressor()
            else:
                self.models[var.name] = RandomForestClassifier()

    def fit(self, X, y):
        name2ind = {var.name: i
                    for i, var in enumerate(model_2_1.variables)}
        for var in model_2_1.variables:
            if not isinstance(var, gadma.DynamicVariable):
                self.models[var.name].fit(X, y[:, name2ind[var.name]])
            else:
                self.models[var.name].fit(X, np.array(y[:, name2ind[var.name]], dtype=str))
            

    def predict(self, X):
        y_pred = {}
        for var in model_2_1.variables:
            y_pred[var.name] = self.models[var.name].predict(X)

        final_y_pred = []
        for _i in range(len(y_pred["nu11"])):
            _y = []
            for var in model_2_1.variables:
                _y.append(y_pred[var.name][_i])
            final_y_pred.append(_y)
        return final_y_pred

    def predict_for_1_1_structure(self, X):
        # 1) predict nu11, t1, d11
        y_pred = np.array(self.predict(X))
        return y_pred[:, 3:]

- Train model

In [13]:
model1 = MyRFFor21ModelIndependent()
model1.fit(X_train_RF, y_train_RF)

- OR load saved model

In [ ]:
filename = 'RandomForestIndependent.joblib'
model1 = joblib.load(filename)

- Get measures on test dataset

In [14]:
y_pred = model1.predict(X_test)

print("Random Forest model 1 with independent estimators for each demographic parameter.")
print_measures(y_true=y_test, y_pred=y_pred)

Random Forest model 1 with independent estimators for each demographic parameter.
Mean squared error: 11.76497710965169
Mean accuracy score: 0.47080000000000005


- Test prediction for our example

In [15]:
y = np.array(model1.predict([test_x]), dtype=object)
y = dynamics2classes(y, back=True)[0]

print("Best known parameters:")
print(best_params)
print("Best known log-likelihood:")
print(best_ll)

print("\nPredicted parameters:")
y_pred = {var.name: value for var, value in zip(model_2_1.variables, y)}
print(y_pred)
print("Log-likelihood:")
print(engine.evaluate(y_pred))

Best known parameters:
{'t1': 1.0, 'nu11': 1.0, 'dyn11': 'Sud', 's1': 0.5, 't2': 0.05, 'nu21': 1.0, 'nu22': 0.1, 'dyn21': 'Sud', 'dyn22': 'Sud', 'm2_12': 5.0, 'm2_21': 2.5}
Best known log-likelihood:
-1310.9310340005

Predicted parameters:
{'t1': 0.36776761929062224, 'nu11': 1.4615867184694744, 'dyn11': 'Sud', 's1': 0.6079165202857019, 't2': 0.272524283820689, 'nu21': 1.747355004224297, 'nu22': 0.30145522192942553, 'dyn21': 'Exp', 'dyn22': 'Exp', 'm2_12': 4.017957245373616, 'm2_21': 1.21407200575233}
Log-likelihood:
-3688.6917267297677


### Model 2. Dependent parameter prediction

Now we add some additional dependency as parameters have them.

![title](model_pictures/model_2_scheme.png)

In [16]:
class MyRFFor21ModelDependent(object):
    def __init__(self):
        # We create our models
        # We have RF regressors for continous variables and Classifiers for dynamics
        self.models = {}
        for var in model_2_1.variables:
            if not isinstance(var, gadma.DynamicVariable):
                self.models[var.name] = RandomForestRegressor()
            else:
                self.models[var.name] = RandomForestClassifier()

    def _add_values(self, X, y, var_names):
        name2ind = {var.name: i
                    for i, var in enumerate(model_2_1.variables)}
        new_X = []
        for _x, _y in zip(X, y):
            new_X.append(_x.tolist())
            for name in var_names:
                new_X[-1].append(_y[name2ind[name]])
        return np.array(new_X, dtype=object)

    def fit(self, X, y):
        # 0) get indices for var names
        name2ind = {var.name: i
                    for i, var in enumerate(model_2_1.variables)}
        # 1) RF for nu11, t1 and dyn11 are independent
        self.models["nu11"].fit(X, y[:, name2ind["nu11"]])
        self.models["t1"].fit(X, y[:, name2ind["t1"]])
        self.models["dyn11"].fit(X, np.array(y[:, name2ind["dyn11"]], dtype=str))

        # 2) Split fraction, times, sizes and dynamics after split depend
        # on those before split
        X_with_prev_values_1 = self._add_values(
            X=X, y=y, var_names=["nu11", "t1", "dyn11"])
        self.models["s1"].fit(X_with_prev_values_1, y[:, name2ind["s1"]])
        self.models["nu21"].fit(X_with_prev_values_1, y[:, name2ind["nu21"]])
        self.models["nu22"].fit(X_with_prev_values_1, y[:, name2ind["nu22"]])
        self.models["t2"].fit(X_with_prev_values_1, y[:, name2ind["t2"]])
        self.models["dyn21"].fit(X_with_prev_values_1, np.array(y[:, name2ind["dyn21"]], dtype=str))
        self.models["dyn22"].fit(X_with_prev_values_1, np.array(y[:, name2ind["dyn22"]], dtype=str))

        # 3) The migrations depends on time, sizes and dynamics
        X_with_prev_values_2 = self._add_values(
            X=X_with_prev_values_1,
            y=y,
            var_names=["s1", "nu21", "nu22", "t2", "dyn21", "dyn22"],
        )
        self.models["m2_21"].fit(X_with_prev_values_2, y[:, name2ind["m2_21"]])
        self.models["m2_12"].fit(X_with_prev_values_2, y[:, name2ind["m2_12"]])

    def predict(self, X):
        # 1) predict nu11, t1, d11
        y_pred = {}
        y_pred["nu11"] = self.models["nu11"].predict(X)
        y_pred["t1"] = self.models["t1"].predict(X)
        y_pred["dyn11"] = self.models["dyn11"].predict(X)
        
        # 2)
        new_X = []
        for i, x in enumerate(X):
            new_X.append(x.tolist())
            new_X[-1].extend([y_pred["nu11"][i], y_pred["t1"][i], y_pred["dyn11"][i]])
        new_X = np.array(new_X)
        y_pred["s1"] = self.models["s1"].predict(new_X)
        y_pred["nu21"] = self.models["nu21"].predict(new_X)
        y_pred["nu22"] = self.models["nu22"].predict(new_X)
        y_pred["t2"] = self.models["t2"].predict(new_X)
        y_pred["dyn21"] = self.models["dyn21"].predict(new_X)
        y_pred["dyn22"] = self.models["dyn22"].predict(new_X)

        # 3)
        new_new_X = []
        for i, x in enumerate(new_X):
            new_new_X.append(x.tolist())
            new_new_X[-1].extend([
                y_pred["s1"][i], y_pred["nu21"][i], y_pred["nu22"][i],
                y_pred["t2"][i], y_pred["dyn21"][i], y_pred["dyn22"][i]]
            )

        y_pred["m2_21"] = self.models["m2_21"].predict(new_new_X)
        y_pred["m2_12"] = self.models["m2_12"].predict(new_new_X)

        final_y_pred = []
        for _i in range(len(y_pred["nu11"])):
            _y = []
            for var in model_2_1.variables:
                _y.append(y_pred[var.name][_i])
            final_y_pred.append(_y)
        return final_y_pred

    def predict_for_1_1_structure(self, X):
        # 1) predict nu11, t1, d11
        y_pred = {}
        y_pred["nu11"] = [1.0 for _ in X]
        y_pred["t1"] = [model_2_1.variables[0].resample() for _ in X]
        y_pred["dyn11"] = ["0" for _ in X]
        
        # 2)
        new_X = []
        for i, x in enumerate(X):
            new_X.append(x.tolist())
            new_X[-1].extend([y_pred["nu11"][i], y_pred["t1"][i], y_pred["dyn11"][i]])
        new_X = np.array(new_X, dtype=object)
        y_pred["s1"] = self.models["s1"].predict(new_X)
        y_pred["nu21"] = self.models["nu21"].predict(new_X)
        y_pred["nu22"] = self.models["nu22"].predict(new_X)
        y_pred["t2"] = self.models["t2"].predict(new_X)
        y_pred["dyn21"] = self.models["dyn21"].predict(new_X)
        y_pred["dyn22"] = self.models["dyn22"].predict(new_X)

        # 3)
        new_new_X = []
        for i, x in enumerate(new_X):
            new_new_X.append(x.tolist())
            new_new_X[-1].extend([
                y_pred["s1"][i], y_pred["nu21"][i], y_pred["nu22"][i],
                y_pred["t2"][i], y_pred["dyn21"][i], y_pred["dyn22"][i]]
            )

        y_pred["m2_21"] = self.models["m2_21"].predict(new_new_X)
        y_pred["m2_12"] = self.models["m2_12"].predict(new_new_X)

        final_y_pred = []
        for _i in range(len(y_pred["nu11"])):
            _y = []
            for var in model_2_1.variables[3:]:
                _y.append(y_pred[var.name][_i])
            final_y_pred.append(_y)
        return final_y_pred

- Train model

In [17]:
model2 = MyRFFor21ModelDependent()
model2.fit(X_train_RF, y_train_RF)

- OR load saved model

In [ ]:
filename = 'RandomForestDependent.joblib'
model2 = joblib.load(filename)

- Get measures on test dataset

In [18]:
y_pred = model2.predict(X_test)

print("Random Forest model 2 with dependent estimators for demographic parameters.")
print_measures(y_true=y_test, y_pred=y_pred)

Random Forest model 2 with dependent estimators for demographic parameters.
Mean squared error: 11.773198395358822
Mean accuracy score: 0.4727333333333334


- Test prediction for our example

In [19]:
y = np.array(model2.predict([test_x]), dtype=object)
y = dynamics2classes(y, back=True)[0]

print("Best known parameters:")
print(best_params)
print("Best known log-likelihood:")
print(best_ll)

print("\nPredicted parameters:")
y_pred = {var.name: value for var, value in zip(model_2_1.variables, y)}
print(y_pred)
print("Log-likelihood:")
print(engine.evaluate(y_pred))

Best known parameters:
{'t1': 1.0, 'nu11': 1.0, 'dyn11': 'Sud', 's1': 0.5, 't2': 0.05, 'nu21': 1.0, 'nu22': 0.1, 'dyn21': 'Sud', 'dyn22': 'Sud', 'm2_12': 5.0, 'm2_21': 2.5}
Best known log-likelihood:
-1310.9310340005

Predicted parameters:
{'t1': 0.3850696776653348, 'nu11': 1.0786090861269844, 'dyn11': 'Sud', 's1': 0.5213370393537992, 't2': 0.18435417793988038, 'nu21': 2.297015285593226, 'nu22': 0.32315403761031564, 'dyn21': 'Exp', 'dyn22': 'Sud', 'm2_12': 2.9904144466727143, 'm2_21': 2.0296450724393207}
Log-likelihood:
-4310.380208183418


### Model 3. Random forest with multi-output predictions

Random forest itself could predict vectors. So we will separate regression and classification and just ask them to predict several values instead one.

![title](model_pictures/model_3_scheme.png)

In [20]:
class MyRFFor21ModelMultiOutput(object):
    def __init__(self):
        # We create our models
        # We have RF regressor for continous variables and Classifier for dynamics
        self.models = [
            RandomForestRegressor(),
            RandomForestClassifier(),
        ]

    def fit(self, X, y):
        is_cont = np.array([not isinstance(var, gadma.DynamicVariable) for var in model_2_1.variables])
        is_disc = np.array([not cont for cont in is_cont])
        
        y_cont = [np.array(_y)[is_cont] for _y in y]
        y_disc = [np.array(_y)[is_disc] for _y in y]
        
        self.models[0].fit(X, y_cont)
        self.models[1].fit(X, y_disc)

    def predict(self, X):
        is_cont = np.array([not isinstance(var, gadma.DynamicVariable) for var in model_2_1.variables])
        is_disc = np.array([not cont for cont in is_cont])
        
        y_pred = np.zeros(shape=(len(X), len(model_2_1.variables)), dtype=object)
        
        y_pred[:, is_cont] = self.models[0].predict(X)
        y_pred[:, is_disc] = self.models[1].predict(X)
        return y_pred

    def predict_for_1_1_structure(self, X):
        # 1) predict nu11, t1, d11
        y_pred = np.array(self.predict(X))
        return y_pred[:, 3:]

- Train model

In [21]:
model3 = MyRFFor21ModelMultiOutput()
model3.fit(X_train_RF, y_train_RF)

- OR load saved model

In [ ]:
filename = 'RandomForestMultiOutput.joblib'
model3 = joblib.load(filename)

- Get measures on test dataset

In [22]:
y_pred = model3.predict(X_test)

print("Random Forest model 3 with multi-output estimators.")
print_measures(y_true=y_test, y_pred=y_pred)

Random Forest model 3 with multi-output estimators.
Mean squared error: 11.898778394692737
Mean accuracy score: 0.4753166666666666


- Test prediction for our example

In [23]:
y = np.array(model3.predict([test_x]), dtype=object)
y = dynamics2classes(y, back=True)[0]

print("Best known parameters:")
print(best_params)
print("Best known log-likelihood:")
print(best_ll)

print("\nPredicted parameters:")
y_pred = {var.name: value for var, value in zip(model_2_1.variables, y)}
print(y_pred)
print("Log-likelihood:")
print(engine.evaluate(y_pred))

Best known parameters:
{'t1': 1.0, 'nu11': 1.0, 'dyn11': 'Sud', 's1': 0.5, 't2': 0.05, 'nu21': 1.0, 'nu22': 0.1, 'dyn21': 'Sud', 'dyn22': 'Sud', 'm2_12': 5.0, 'm2_21': 2.5}
Best known log-likelihood:
-1310.9310340005

Predicted parameters:
{'t1': 0.26863903612766166, 'nu11': 1.678640535125808, 'dyn11': 'Sud', 's1': 0.6216949404299615, 't2': 0.27383689275649287, 'nu21': 1.7011780079568388, 'nu22': 0.5246284441325718, 'dyn21': 'Exp', 'dyn22': 'Exp', 'm2_12': 3.421735042716503, 'm2_21': 1.1691820428019235}
Log-likelihood:
-8114.9975080088625


### Save trained models

In [24]:
filename = 'RandomForestIndependent.joblib'
joblib.dump(model1, filename, compress=9)

['RandomForestIndependent.joblib']

In [25]:
filename = 'RandomForestDependent.joblib'
joblib.dump(model2, filename, compress=9)

['RandomForestDependent.joblib']

In [26]:
filename = 'RandomForestMultiOutput.joblib'
joblib.dump(model3, filename, compress=9)

['RandomForestMultiOutput.joblib']

## Convolution Neural Network (CNN)

Finally we want to use CNN to predict the same demographic parameters from allele frequency spectrum.

CNN require much more examples to train so we will give it all 200,000 samples we have. Usually pictures are much bigger, we have only 6x6 pixels. As we have small picture then we have very simple network. 

#### Install and import libraries

In [ ]:
!pip install keras
!pip install tensorflow

In [32]:
CUDA_VISIBLE_DEVICES=""
from keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Lambda
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.models import model_from_json
from tensorflow.keras.utils import to_categorical
from gadma.utils.variables import DynamicVariable
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

### Model. Independent predictions

The CNN model is simple. Each parameter is predicted with its own CNN. The convolutions are small: 2x2 size. Number of filters increases: 16, 32 and 64.The final full-connected layer has 8 neurons. The final layer depends on type pf parameter. If it is continous then there is last fully connected (FC) layer that returns 1 output for sigmoid activation. If it is discrete then the last layer is also fully connected but returns 3 outputs (3 classes) for softmax activation.

The common part of CNN for parameters looks like:

![title](model_pictures/cnn_common_scheme.png)

We set the size of input image and create special custom activation function:

In [28]:
IM_WIDTH = IM_HEIGHT = ns_per_pop + 1


def get_custom_activation(bounds):
    def custom_activation(x):
        return (K.sigmoid(x) * (bounds[1] - bounds[0])) + bounds[0]
    return custom_activation

Create data generator for neural network

In [29]:
class DataGenerator():
    """
    Data generator for the UTKFace dataset. This class should be used when training our Keras multi-output model.
    """ 
    def generate_images(self, X, y, is_training, batch_size=16):
        """
        Used to generate a batch with images when training/testing/validating our Keras model.
        """
        X_resized = np.array(X)
        X_resized = np.array([np.reshape(np.array(x), (-1, 6, 1)) for x in X_resized])

        # arrays to store our batched data
        images = []
        answers = [[] for _ in model_2_1.variables]
        while True:
            for idx in range(len(X)):
                im = X_resized[idx]
                for i in range(len(model_2_1.variables)):
                    var = model_2_1.variables[i]
                    if isinstance(var, gadma.DynamicVariable):
                        answers[i].append(to_categorical(int(y[idx][i]), 3))
                    else:
                        answers[i].append(float(y[idx][i]))
                images.append(im)
                
                # yielding condition
                if len(images) >= batch_size:
                    yield np.array(images), [np.array(answ) for answ in answers]
                    images = []
                    answers = [[] for _ in model_2_1.variables]
                    
            if not is_training:
                break
                
data_generator = DataGenerator()

And function to convert CNN output to output of RF

In [30]:
def convert_cnn_output_to_rf_output(cnn_output):
    rf_output = []
    for i in range(len(cnn_output[0])):
        out = []
        for j, var in enumerate(model_2_1.variables):
            if isinstance(var, gadma.DynamicVariable):
                argmax = max(list(range(3)), key=lambda t: cnn_output[j][i][t])
                out.append(str(argmax))
            else:
                out.append(cnn_output[j][i][0])
        rf_output.append(np.array(out, dtype=object))
    return rf_output

- Create model with `Keras`

In [33]:
class KerasModelIndependent():
    def make_default_hidden_layers(self, inputs):
        filters = (16, 32, 64)
        for (i, f) in enumerate(filters):
            if i == 0: x = inputs
            x = Conv2D(f, (2, 2), padding="same", activation="relu")(x)
            x = BatchNormalization(axis=-1)(x)
            if i == (len(filters) - 1):
                x = MaxPooling2D(pool_size=(2, 2))(x)
            x = Dropout(0.25)(x)
        return x

    def build_float_branch(self, inputs, name, bounds):
        x = self.make_default_hidden_layers(inputs)
        # flatten the volume, then FC => RELU => BN => DROPOUT
        x = Flatten()(x)
        x = Dense(8, activation="relu")(x)
        # we have regression
        x = Dense(1)(x)
        x = Activation(get_custom_activation(bounds), name=name)(x)
        return x

    def build_discrete_branch(self, inputs, name):
        x = self.make_default_hidden_layers(inputs)
        # flatten the volume, then FC => RELU => BN => DROPOUT
        x = Flatten()(x)
        x = Dense(8, activation="relu")(x)
        x = Dense(3)(x)
        x = Activation("softmax", name=name)(x)
        return x

    def assemble_full_model(self, width, height):
        """
        Used to assemble our multi-output model CNN.
        """
        input_shape = (height, width, 1)
        inputs = Input(shape=input_shape)
        branches = []
        for var in model_2_1.variables:
            if isinstance(var, gadma.DynamicVariable):
                branches.append(self.build_discrete_branch(inputs, name=f"{var.name}_output"))
            else:
                branches.append(self.build_float_branch(inputs, name=f"{var.name}_output", bounds=var.domain))
        model = Model(inputs=inputs,
                     outputs = branches,
                     name="keras_net")
        return model
    
model = KerasModelIndependent().assemble_full_model(IM_WIDTH, IM_HEIGHT)
# model.summary()

#### Create validation dataset

In [34]:
X_train_CNN, X_val_CNN, y_train_CNN, y_val_CNN = tr(
    X_train, y_train, train_size = 0.8, random_state=500, shuffle=True
)

#### Training

- Train model

In [35]:
init_lr = 1e-4
epochs = 100
opt = Adam(learning_rate=init_lr, decay=init_lr / epochs)
model.compile(optimizer=opt, 
              loss={
                  f"{var.name}_output": "mse" if not isinstance(var, gadma.DynamicVariable) else "categorical_crossentropy"
                  for var in model_2_1.variables
              },
              metrics={
                  f"{var.name}_output": "mae" if not isinstance(var, gadma.DynamicVariable) else "categorical_accuracy"
                  for var in model_2_1.variables
              })

batch_size = 32
valid_batch_size = 32
train_gen = data_generator.generate_images(X_train_CNN, y_train_CNN, is_training=True, batch_size=batch_size)
valid_gen = data_generator.generate_images(X_val_CNN, y_val_CNN, is_training=True, batch_size=valid_batch_size)
callbacks = [
    ModelCheckpoint("./model_checkpoint", monitor='val_loss')
]
history = model.fit(train_gen,
                    steps_per_epoch=len(X_train)//batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=len(X_test)//valid_batch_size)

Epoch 1/100


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2498/2500 [============================>.] - ETA: 0s - loss: 162.4623 - t1_output_loss: 1.1289 - nu11_output_loss: 44.0866 - dyn11_output_loss: 0.9834 - s1_output_loss: 0.0857 - t2_output_loss: 1.2808 - nu21_output_loss: 52.0145 - nu22_output_loss: 52.6293 - dyn21_output_loss: 1.1006 - dyn22_output_loss: 1.1097 - m2_12_output_loss: 4.3464 - m2_21_output_loss: 3.6963 - t1_output_mae: 0.7348 - nu11_output_mae: 2.4305 - dyn11_output_categorical_accuracy: 0.6194 - s1_output_mae: 0.2522 - t2_output_mae: 0.8007 - nu21_output_mae: 2.9695 - nu22_output_mae: 3.0115 - dyn21_output_categorical_accuracy: 0.4354 - dyn22

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2500/2500 [==============================] - 54s 22ms/step - loss: 76.6789 - t1_output_loss: 0.7615 - nu11_output_loss: 22.2224 - dyn11_output_loss: 0.9025 - s1_output_loss: 0.0764 - t2_output_loss: 0.7593 - nu21_output_loss: 24.5645 - nu22_output_loss: 22.9067 - dyn21_output_loss: 1.0517 - dyn22_output_loss: 1.0615 - m2_12_output_loss: 1.1886 - m2_21_output_loss: 1.1836 - t1_output_mae: 0.5757 - nu11_output_mae: 1.8540 - dyn11_output_categorical_accuracy: 0.6306 - s1_output_mae: 0.2363 - t2_output_mae: 0.5676 - nu21_output_mae: 1.8504 - nu22_output_mae: 1.8284 - dyn21_output_categorical_accuracy: 0.4473 - dyn22_output_categorical_accuracy: 0.4436 - m2_12_output_mae: 0.6531 - m2_21_output_mae: 0.6394 - val_loss: 77.8436 - val_t1_output_loss: 0.6998 - val_nu11_output_loss: 23.7950 - val_dyn11_output_loss: 0.9128 - val_s1_output_loss: 0.0763 - val_t2_output_loss: 0.6850 - val_nu21_output_loss: 23.5128 - val_nu22_output_loss: 23.2786 - val_dyn21_output_loss: 1.0532 - val_dyn22_output_loss

- OR load saved weights

In [ ]:
model.load_weights("model_cnn_weights.h5")

- Get measures on test dataset

In [36]:
X_test_resized = np.array([np.reshape(np.array(x), (-1, 6, 1)) for x in X_test])
y_pred = model.predict(X_test_resized)
y_pred = convert_cnn_output_to_rf_output(y_pred)

print("Convolutional neural network with independent branches.")
print_measures(y_true=y_test, y_pred=y_pred)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
625/625 [==============================] - 4s 5ms/step
Convolutional neural network with independent branches.
Mean squared error: 8.93617170301409
Mean accuracy score: 0.5061


- Test prediction for our example

In [37]:
y = model.predict(np.array([np.reshape(test_x, (6, 6, 1))]))
y = convert_cnn_output_to_rf_output(y)
y = dynamics2classes(y, back=True)[0]

print("Best known parameters:")
print(best_params)
print("Best known log-likelihood:")
print(best_ll)

print("\nPredicted parameters:")
y_pred = {var.name: value for var, value in zip(model_2_1.variables, y)}
print(y_pred)
print("Log-likelihood:")
print(engine.evaluate(y_pred))

1/1 [==============================] - 1s 686ms/step
Best known parameters:
{'t1': 1.0, 'nu11': 1.0, 'dyn11': 'Sud', 's1': 0.5, 't2': 0.05, 'nu21': 1.0, 'nu22': 0.1, 'dyn21': 'Sud', 'dyn22': 'Sud', 'm2_12': 5.0, 'm2_21': 2.5}
Best known log-likelihood:
-1310.9310340005

Predicted parameters:
{'t1': 0.4538623, 'nu11': 1.2582624, 'dyn11': 'Sud', 's1': 0.5809103, 't2': 0.272153, 'nu21': 1.8804532, 'nu22': 0.21437056, 'dyn21': 'Sud', 'dyn22': 'Sud', 'm2_12': 4.303458, 'm2_21': 0.8473885}
Log-likelihood:
-6813.690614199338


In [38]:
# save model weights
model.save_weights("model_cnn_weights.h5")